In [4]:
%display latex

In [5]:
M = Manifold(4, 'M')
print(M)

4-dimensional differentiable manifold M


In [6]:
X.<t,x,th,ph> = M.chart(r't:\tau x:(0,+oo):\chi th:(0,pi):\theta ph:(0,2*pi):\phi')
X

Chart (M, (t, x, th, ph))

In [8]:
g = M.lorentzian_metric('g')
m0 = var('m_0', domain='real'); assume(m0>0)
xs = var('xs', latex_name=r'\chi_{\rm s}', domain='real'); assume(xs>0)
a = (1 - 3*sqrt(m0/(2*xs^3)) * t)^(2/3)
g[0,0] = -1
g[1,1] = a^2
g[2,2] = (a*x)^2
g[3,3] = (a*x*sin(th))^2
g.display()

g = -dt*dt + (-3*sqrt(1/2)*t*sqrt(m_0/xs^3) + 1)^(4/3) dx*dx + (-3*sqrt(1/2)*t*sqrt(m_0/xs^3) + 1)^(4/3)*x^2 dth*dth + (-3*sqrt(1/2)*t*sqrt(m_0/xs^3) + 1)^(4/3)*x^2*sin(th)^2 dph*dph

In [9]:
g.display_comp()

g_t,t = -1 
g_x,x = (-3*sqrt(1/2)*t*sqrt(m_0/xs^3) + 1)^(4/3) 
g_th,th = (-3*sqrt(1/2)*t*sqrt(m_0/xs^3) + 1)^(4/3)*x^2 
g_ph,ph = (-3*sqrt(1/2)*t*sqrt(m_0/xs^3) + 1)^(4/3)*x^2*sin(th)^2

In [10]:
Ric = g.ricci()
print(Ric)

Field of symmetric bilinear forms Ric(g) on the 4-dimensional differentiable manifold M


In [11]:
Ric.display_comp()

Ric(g)_t,t = -6*m_0/(6*sqrt(2)*sqrt(m_0)*t*xs^(3/2) - 9*m_0*t^2 - 2*xs^3) 
Ric(g)_x,x = 3*2^(2/3)*m_0/((-3*sqrt(2)*sqrt(m_0)*t + 2*xs^(3/2))^(2/3)*xs^2) 
Ric(g)_th,th = 3*2^(2/3)*m_0*x^2/((-3*sqrt(2)*sqrt(m_0)*t + 2*xs^(3/2))^(2/3)*xs^2) 
Ric(g)_ph,ph = 3*2^(2/3)*m_0*x^2*sin(th)^2/((-3*sqrt(2)*sqrt(m_0)*t + 2*xs^(3/2))^(2/3)*xs^2)

In [12]:
Ric.display()

Ric(g) = -6*m_0/(6*sqrt(2)*sqrt(m_0)*t*xs^(3/2) - 9*m_0*t^2 - 2*xs^3) dt*dt + 3*2^(2/3)*m_0/((-3*sqrt(2)*sqrt(m_0)*t + 2*xs^(3/2))^(2/3)*xs^2) dx*dx + 3*2^(2/3)*m_0*x^2/((-3*sqrt(2)*sqrt(m_0)*t + 2*xs^(3/2))^(2/3)*xs^2) dth*dth + 3*2^(2/3)*m_0*x^2*sin(th)^2/((-3*sqrt(2)*sqrt(m_0)*t + 2*xs^(3/2))^(2/3)*xs^2) dph*dph

In [13]:
Ric[0,0]

-6*m_0/(6*sqrt(2)*sqrt(m_0)*t*xs^(3/2) - 9*m_0*t^2 - 2*xs^3)

In [14]:
G = Ric - 1/2*g.ricci_scalar() * g
G.set_name('G')
print(G)

Field of symmetric bilinear forms G on the 4-dimensional differentiable manifold M


In [15]:
G.display_comp()

G_t,t = -12*m_0/(6*sqrt(2)*sqrt(m_0)*t*xs^(3/2) - 9*m_0*t^2 - 2*xs^3)

In [16]:
u = M.vector_field('u')
u[0] = 1
u.display()

u = d/dt

In [17]:
g(u,u).display()

g(u,u): M --> R
   (t, x, th, ph) |--> -1

In [18]:
u_form = u.down(g)
print(u_form)

1-form on the 4-dimensional differentiable manifold M


In [19]:
u_form.display()

-dt

In [20]:
rho = function('rho')
T = rho(t,x)* (u_form * u_form)
T.set_name('T')
print(T)

Field of symmetric bilinear forms T on the 4-dimensional differentiable manifold M


In [21]:
T.display()

T = rho(t, x) dt*dt

In [22]:
E = G - 8*pi*T
E.set_name('E')
print(E)

Field of symmetric bilinear forms E on the 4-dimensional differentiable manifold M


In [23]:
E.display()

E = -4*(12*sqrt(2)*pi*sqrt(m_0)*t*xs^(3/2)*rho(t, x) - 18*pi*m_0*t^2*rho(t, x) - 4*pi*xs^3*rho(t, x) + 3*m_0)/(6*sqrt(2)*sqrt(m_0)*t*xs^(3/2) - 9*m_0*t^2 - 2*xs^3) dt*dt

In [24]:
E.display_comp()

E_t,t = -4*(12*sqrt(2)*pi*sqrt(m_0)*t*xs^(3/2)*rho(t, x) - 18*pi*m_0*t^2*rho(t, x) - 4*pi*xs^3*rho(t, x) + 3*m_0)/(6*sqrt(2)*sqrt(m_0)*t*xs^(3/2) - 9*m_0*t^2 - 2*xs^3)

In [25]:
E[0,0]

-4*(12*sqrt(2)*pi*sqrt(m_0)*t*xs^(3/2)*rho(t, x) - 18*pi*m_0*t^2*rho(t, x) - 4*pi*xs^3*rho(t, x) + 3*m_0)/(6*sqrt(2)*sqrt(m_0)*t*xs^(3/2) - 9*m_0*t^2 - 2*xs^3)

In [26]:
eq = (-E[0,0]/4).expr().numerator() == 0
eq

12*sqrt(2)*pi*sqrt(m_0)*t*xs^(3/2)*rho(t, x) - 18*pi*m_0*t^2*rho(t, x) - 4*pi*xs^3*rho(t, x) + 3*m_0 == 0

In [27]:
solve(eq, rho(t,x))

[rho(t, x) == -3/2*m_0/(6*sqrt(2)*pi*sqrt(m_0)*t*xs^(3/2) - 9*pi*m_0*t^2 - 2*pi*xs^3)]